# **How to Query The Graph Protocol for On-Chain Data using Python**
### *A step-by-step guide on querying on-chain data from Subgraphs using Python, GraphQL, and Subgrounds.*

In this tutorial, we will query the [ENS Subgraph](https://thegraph.com/hosted-service/subgraph/ensdomains/ens) using two methods; raw [GraphQL](https://thegraph.com/docs/en/querying/graphql-api/) query and [Subgrounds](https://playgrounds-analytics.gitbook.io/playgrounds-docs/subgrounds/the-basics) library by [Playgrounds](https://www.playgrounds.network/).

The goal is for you to be able to:

- query any Subgraph data using python
- understand the two querying methods; using raw GraphQL and Subgrounds library

## Using Raw GraphQL Query Subgraphs with Python

In [129]:
import pandas as pd
import requests

# variable holding the query payload
query = """
{
    registrations(first:5, orderBy:registrationDate, orderDirection:desc){
        domain{
            name
        }
        registrant {
            id
        }
        registrationDate
        cost
        expiryDate
    }
}
"""

def get_data(query):
    """This function posts a request to make API call to ENS Subgraph URL
    
    parameter:
    ----------
    query: payload to make request for data
    
    return:
    -------
    JSON: ENS registration data
    """
    
    # Send request for data
    response = requests.post('https://api.thegraph.com/subgraphs/name/ensdomains/ens'
                             '',
                             json={"query":query})

    # Code 200 means no errors
    if response.status_code == 200:  
        return response.json()
    # If errors, print the error code for debugging
    else: 
        raise Exception("Query failed with return code {}".format(response.staus_code))
        
# Get data and display output
data = get_data(query)

# Display data
data

{'data': {'registrations': [{'domain': {'name': '_kingcharlesiii.eth'},
    'registrant': {'id': '0xf66a3457a32e246a55b8626b647f44892a2a1510'},
    'registrationDate': '1662666448',
    'cost': '3065861783686301',
    'expiryDate': '1694223400'},
   {'domain': {'name': 'mythicalanon.eth'},
    'registrant': {'id': '0xcb8ddb86766111f72b1a067ea05902c07d01c508'},
    'registrationDate': '1662666448',
    'cost': '9197585351058903',
    'expiryDate': '1757337304'},
   {'domain': {'name': 'cadu305.eth'},
    'registrant': {'id': '0x71851be2a86081373f746826223a6f551e6d68d1'},
    'registrationDate': '1662666448',
    'cost': '3065861783686301',
    'expiryDate': '1694223400'},
   {'domain': {'name': '_queenelizabethii.eth'},
    'registrant': {'id': '0xf66a3457a32e246a55b8626b647f44892a2a1510'},
    'registrationDate': '1662666448',
    'cost': '3065861783686301',
    'expiryDate': '1694223400'},
   {'domain': {'name': 'm4w.eth'},
    'registrant': {'id': '0x85f0398fdbd02bbe49985ecabbbfc3ea2

##  Using Subgrounds Python Library to Query Subgraphs

In [130]:
from subgrounds import Subgrounds
from subgrounds.subgraph import SyntheticField
from datetime import datetime

# Instantiate Subgrounds
sg = Subgrounds()

# ENS http link from The Graph Hosted Service
ens = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/ensdomains/ens')

# synthetic field to convert UNIX dates to timestamp
# registrationdate synthetic field
ens.Registration.registrationdate = SyntheticField(
    lambda registrationDate: str(datetime.fromtimestamp(registrationDate)),
    SyntheticField.STRING,
    ens.Registration.registrationDate
)

# expirydate synthetic field
ens.Registration.expirydate = SyntheticField(
    lambda expiryDate: str(datetime.fromtimestamp(expiryDate)),
    SyntheticField.STRING,
    ens.Registration.expiryDate
)

# Select the latest 1000 registration names by registration datetime
registrations = ens.Query.registrations(
    first=1000, # latest 1000 registrations
    orderBy=ens.Registration.registrationDate, # order registrations by time
    orderDirection="desc" # latest registration data will be first
)
 
field_paths = [
    registrations.domain.name, # ens domain like "vitalik.eth"
    registrations.registrant.id, # hexadecimal eth address
    registrations.registrationdate, # time in epoch format
    registrations.cost, # price for registration
    registrations.expirydate # expiry date of domain
]

# Get data 
df = sg.query_df(field_paths)

# Create column with registration cost converted from wei to ether
df['registrations_cost'] = df['registrations_cost'] / (10**18)

# Rename columns for simplicity
df = df.rename(columns={'registrations_domain_name': 'ens_name',
                        'registrations_registrant_id': 'owner_address',
                        'registrations_registrationdate': 'registration_date',
                        'registrations_cost': 'registration_cost_ether',
                        'registrations_expirydate': 'expiry_date'
                        })

# Display the first 5 data points
df.head()

,ens_name,owner_address,registration_date,registration_cost_ether,expiry_date
0,_kingcharlesiii.eth,0xf66a3457a32e246a55b8626b647f44892a2a1510,2022-09-08 15:47:28,0.003066,2023-09-08 21:36:40
1,mythicalanon.eth,0xcb8ddb86766111f72b1a067ea05902c07d01c508,2022-09-08 15:47:28,0.009198,2025-09-08 09:15:04
2,cadu305.eth,0x71851be2a86081373f746826223a6f551e6d68d1,2022-09-08 15:47:28,0.003066,2023-09-08 21:36:40
3,_queenelizabethii.eth,0xf66a3457a32e246a55b8626b647f44892a2a1510,2022-09-08 15:47:28,0.003066,2023-09-08 21:36:40
4,m4w.eth,0x85f0398fdbd02bbe49985ecabbbfc3ea2d192a3c,2022-09-08 15:47:28,0.392430,2023-09-08 21:36:40
